In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE131027"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE131027"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE131027.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE131027.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE131027.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"High frequency of pathogenic germline variants in genes associated with homologous recombination repair in patients with advanced solid cancers"
!Series_summary	"We identified pathogenic and likely pathogenic variants in 17.8% of the patients within a wide range of cancer types. In particular, mesothelioma, ovarian cancer, cervical cancer, urothelial cancer, and cancer of unknown primary origin displayed high frequencies of pathogenic variants. In total, 22 BRCA1 and BRCA2 germline variant were identified in 12 different cancer types, of which 10 (45%) variants were not previously identified in these patients. Pathogenic germline variants were predominantly found in DNA repair pathways; approximately half of the variants were within genes involved in homologous recombination repair. Loss of heterozygosity and somatic second hits were identified in several of these genes, supporting possible causality for cancer development. A potential treatment t

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyzing the dataset information

# 1. Gene Expression Data Availability
# Based on the series title and overall design mentioning "expression features",
# this dataset likely contains gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Looking for trait related to Kidney Clear Cell Carcinoma
trait_row = 1  # The row with 'cancer' types includes 'Renal cell carcinoma' which is equivalent to Kidney Clear Cell Carcinoma
age_row = None  # No age information is available in the sample characteristics
gender_row = None  # No gender information is available in the sample characteristics

# 2.2 Data Type Conversion Functions
def convert_trait(trait_value):
    """Convert trait value to binary format (0 for non-KCCC, 1 for KCCC)"""
    if trait_value is None:
        return None
    # Extract the value after the colon
    if ':' in trait_value:
        trait_value = trait_value.split(':', 1)[1].strip()
    # Match 'Renal cell carcinoma' as a positive case for Kidney Clear Cell Carcinoma
    if 'Renal cell carcinoma' in trait_value:
        return 1
    else:
        return 0

def convert_age(age_value):
    """Convert age value to continuous format"""
    # This function won't be used since age data is not available
    if age_value is None:
        return None
    if ':' in age_value:
        age_value = age_value.split(':', 1)[1].strip()
    try:
        return float(age_value)
    except:
        return None

def convert_gender(gender_value):
    """Convert gender value to binary format (0 for female, 1 for male)"""
    # This function won't be used since gender data is not available
    if gender_value is None:
        return None
    if ':' in gender_value:
        gender_value = gender_value.split(':', 1)[1].strip().lower()
    if gender_value in ['female', 'f']:
        return 0
    elif gender_value in ['male', 'm']:
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait data is available
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary
    # The sample characteristics dictionary is from the previous step output
    sample_char_dict = {0: ['tissue: tumor biopsy'], 
                        1: ['cancer: Breast cancer', 'cancer: Colorectal cancer', 'cancer: Bile duct cancer', 
                            'cancer: Mesothelioma', 'cancer: Urothelial cancer', 'cancer: Pancreatic cancer', 
                            'cancer: Melanoma', 'cancer: Hepatocellular carcinoma', 'cancer: Ovarian cancer', 
                            'cancer: Cervical cancer', 'cancer: Head and Neck cancer', 'cancer: Sarcoma', 
                            'cancer: Prostate cancer', 'cancer: Adenoid cystic carcinoma', 'cancer: NSCLC', 
                            'cancer: Oesophageal cancer', 'cancer: Thymoma', 'cancer: Others', 'cancer: CUP', 
                            'cancer: Renal cell carcinoma', 'cancer: Gastric cancer', 
                            'cancer: Neuroendocrine cancer', 'cancer: vulvovaginal'], 
                        2: ['mutated gene: ATR', 'mutated gene: FAN1', 'mutated gene: ERCC3', 'mutated gene: FANCD2', 
                            'mutated gene: BAP1', 'mutated gene: DDB2', 'mutated gene: TP53', 'mutated gene: ATM', 
                            'mutated gene: CHEK1', 'mutated gene: BRCA1', 'mutated gene: WRN', 'mutated gene: CHEK2', 
                            'mutated gene: BRCA2', 'mutated gene: XPC', 'mutated gene: PALB2', 'mutated gene: ABRAXAS1', 
                            'mutated gene: NBN', 'mutated gene: BLM', 'mutated gene: FAM111B', 'mutated gene: FANCA', 
                            'mutated gene: MLH1', 'mutated gene: BRIP1', 'mutated gene: IPMK', 'mutated gene: RECQL', 
                            'mutated gene: RAD50', 'mutated gene: FANCM', 'mutated gene: GALNT12', 'mutated gene: SMAD9', 
                            'mutated gene: ERCC2', 'mutated gene: FANCC'], 
                        3: ['predicted: HRDEXP: HRD', 'predicted: HRDEXP: NO_HRD'], 
                        4: ['parp predicted: kmeans-2: PARP sensitive', 'parp predicted: kmeans-2: PARP insensitive']}
    
    # Convert the dictionary to a pandas DataFrame format that geo_select_clinical_features can process
    # Create empty rows for each sample
    samples = []
    for cancer_type in sample_char_dict[trait_row]:
        sample = {}
        sample[trait_row] = cancer_type
        samples.append(sample)
    
    clinical_data = pd.DataFrame(samples)
    
    # Extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the processed dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the processed clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{1: [0.0]}
Clinical data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE131027.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:


Successfully extracted gene data with 54675 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers observed in the gene expression data (like '1007_s_at', '1053_at', etc.)
# appear to be Affymetrix probe IDs from a microarray platform, not human gene symbols.
# These identifiers need to be mapped to standard gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 5084867 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the columns for probe ID and gene symbol in the gene annotation data
# Based on the preview, we need the 'ID' column for probe identifiers and 'Gene Symbol' for gene symbols
probe_col = 'ID'          # Column containing probe IDs matching gene expression data
gene_symbol_col = 'Gene Symbol'  # Column containing the gene symbols we want to map to

print(f"Using probe column: '{probe_col}' and gene symbol column: '{gene_symbol_col}'")

# 2. Create a mapping dataframe with the probe IDs and gene symbols
try:
    # Use the helper function to extract the mapping from the annotation dataframe
    gene_mapping = get_gene_mapping(gene_annotation, prob_col=probe_col, gene_col=gene_symbol_col)
    
    # Preview the mapping dataframe
    print(f"\nGene mapping dataframe created with {len(gene_mapping)} rows")
    print("First few rows of gene mapping:")
    print(preview_df(gene_mapping))
    
    # 3. Apply the gene mapping to convert probe-level measurements to gene expression data
    # This handles probes mapping to multiple genes by dividing expression and summing by gene
    print("\nApplying gene mapping to convert probe data to gene expression data...")
    gene_data = apply_gene_mapping(expression_df=gene_data, mapping_df=gene_mapping)
    
    # Preview the resulting gene expression data
    print(f"Gene expression data created with {len(gene_data.index)} unique genes")
    print("First few gene symbols and sample columns:")
    print(preview_df(gene_data))
    
    # Save the gene expression data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data.to_csv(out_gene_data_file)
    print(f"Gene expression data saved to {out_gene_data_file}")

except Exception as e:
    print(f"Error in gene mapping process: {e}")
    is_gene_available = False


Using probe column: 'ID' and gene symbol column: 'Gene Symbol'



Gene mapping dataframe created with 45782 rows
First few rows of gene mapping:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}

Applying gene mapping to convert probe data to gene expression data...
Gene expression data created with 21278 unique genes
First few gene symbols and sample columns:
{'GSM3759992': [4.390919088, 4.498580122, 7.712909389, 14.491903516, 6.186830839], 'GSM3759993': [9.637093765, 4.911001093, 17.768013934, 16.222561284999998, 4.286040695], 'GSM3759994': [5.370775767, 4.409248332, 8.704945831, 15.166473245, 5.067773944], 'GSM3759995': [7.376018685, 4.958839994, 14.905013256, 15.598187591999999, 5.807062391], 'GSM3759996': [9.74745451, 4.12673215, 16.923252481, 15.317524529, 3.963854128], 'GSM3759997': [7.568073556, 5.894117526, 7.351391774, 14.574576716, 3.236874339], 'GSM3759998': [12.62778479, 4.571267834, 21.82809339, 17.392582999, 4.999760334], 'GSM3759999': [12.22717927, 4.925

Gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE131027.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    # Check if gene_data is empty before normalization
    if gene_data.empty:
        print("Gene data is empty. Skipping normalization.")
        normalized_gene_data = gene_data
    else:
        normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (even if empty, to maintain consistent workflow)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# Use is_trait_available from previous steps, don't recheck trait_row
print("\nChecking trait availability from previous analysis...")
# Load the clinical data from the previously saved file
try:
    clinical_df = pd.read_csv(out_clinical_data_file)
    print(f"Loaded clinical data with shape: {clinical_df.shape}")
    # Rename the column '1' to the trait name if it exists
    if '1' in clinical_df.columns:
        clinical_df = clinical_df.rename(columns={'1': trait})
        print(f"Renamed column '1' to '{trait}'")
    is_trait_available = trait in clinical_df.columns
except Exception as e:
    print(f"Error loading clinical data: {e}")
    clinical_df = pd.DataFrame()
    is_trait_available = False

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if is_trait_available and not normalized_gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("Clinical columns:", list(clinical_df.columns))
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Transpose clinical data so samples are rows
        if clinical_df.shape[0] == 1:  # If it's currently 1 row with samples as columns
            clinical_df = clinical_df.T
            clinical_df.columns = [trait]
            print(f"Transposed clinical data to shape: {clinical_df.shape}")
        
        # Now link clinical and genetic data
        linked_data = pd.concat([clinical_df, normalized_gene_data.T], axis=1)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        if trait in linked_data.columns:
            trait_count = linked_data[trait].count()
            print(f"Number of samples with trait values: {trait_count}")
            
            if trait_count > 0:
                # 3. Handle missing values systematically
                print("\nHandling missing values...")
                linked_data = handle_missing_values(linked_data, trait)
                print(f"After handling missing values, data shape: {linked_data.shape}")
                
                # Check if we still have samples after missing value handling
                if linked_data.shape[0] > 0:
                    # 4. Determine whether the trait and demographic features are biased
                    print("\nChecking for bias in features...")
                    is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                else:
                    print("Error: All samples were removed during missing value handling.")
                    is_biased = True
            else:
                print("No samples have valid trait values. Dataset cannot be used.")
                is_biased = True
        else:
            print(f"Trait column '{trait}' not found in linked data.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing.")
        if not is_trait_available:
            print(f"Reason: Trait column '{trait}' is not available in clinical data.")
        if normalized_gene_data.empty:
            print("Reason: Gene expression data is empty.")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
note = ""
if 'linked_data' in locals() and linked_data.empty:
    if not normalized_gene_data.empty and not clinical_df.empty:
        note = "Dataset failed in linking phase. Clinical data structure could not be properly matched with gene expression data."
    elif normalized_gene_data.empty:
        note = "Dataset failed in gene mapping phase. Could not properly map probe IDs to gene symbols."
    elif clinical_df.empty:
        note = "Dataset failed in clinical data extraction. No valid trait information available."
elif 'is_biased' in locals() and is_biased:
    note = "Dataset passed initial processing but contains severely biased trait distribution."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=not normalized_gene_data.empty,  # Set based on actual gene data availability
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note=note
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 19845 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE131027.csv

Checking trait availability from previous analysis...
Loaded clinical data with shape: (1, 1)
Renamed column '1' to 'Kidney_Clear_Cell_Carcinoma'

Linking clinical and genetic data...
Clinical columns: ['Kidney_Clear_Cell_Carcinoma']
First few genetic sample columns: ['GSM3759992', 'GSM3759993', 'GSM3759994', 'GSM3759995', 'GSM3759996']
Transposed clinical data to shape: (1, 1)
Linked data shape: (93, 19846)
Number of samples with trait values: 1

Handling missing values...
After handling missing values, data shape: (0, 1)
Error: All samples were removed during missing value handling.

Performing final validation...
Abnormality detected in the cohort: GSE131027. Preprocessing failed.
Dataset not usable for Kidney_Clear_Cell_Carcinoma association studies. Data not saved.
